In [1]:
import os
import json
import sys
from elasticsearch import Elasticsearch
import wikipediaapi
from slugify import slugify

In [2]:
client = Elasticsearch("http://localhost:9200")

In [3]:
wiki_wiki = wikipediaapi.Wikipedia('en')

In [25]:
category = 'Marvel Comics'

In [26]:
def print_categorymembers(categorymembers, level=0, max_level=1):
        for c in categorymembers.values():
            print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
            if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
                print_categorymembers(c.categorymembers, level=level + 1, max_level=max_level)


cat = wiki_wiki.page(f"Category:{category}")
print_categorymembers(cat.categorymembers)

*: History of webcomics (ns: 0)
*: Webcomic (ns: 0)
*: Angry Little Girls (ns: 0)
*: Boxcar Comics (ns: 0)
*: Bruno (webcomic) (ns: 0)
*: Business of webcomics (ns: 0)
*: Chopping Block (ns: 0)
*: Katie Cook (writer) (ns: 0)
*: Demon-cratic Singapore (ns: 0)
*: Diesel Sweeties (ns: 0)
*: Dork Tower (ns: 0)
*: Dumbrella (ns: 0)
*: The Erotic Adventures of Buffy and Evil Vampire Willow (ns: 0)
*: Everyday Blues (ns: 0)
*: Explodingdog (ns: 0)
*: Game Critter Super-Squad! (ns: 0)
*: Gender and webcomics (ns: 0)
*: Goats (webcomic) (ns: 0)
*: Guest comic (ns: 0)
*: HJ-Story (ns: 0)
*: Inkblazers (ns: 0)
*: Keenspot (ns: 0)
*: Marvel CyberComics (ns: 0)
*: Modern Polaxis (ns: 0)
*: New England Webcomics Weekend (ns: 0)
*: Once Upon a Saturday (ns: 0)
*: Platinum Grit (ns: 0)
*: The Really Heavy Greatcoat (ns: 0)
*: Roger Cosmonkey (ns: 0)
*: Rogues of Clwyd-Rhan (ns: 0)
*: Saturnalia (webcomic) (ns: 0)
*: Savage Chickens (ns: 0)
*: ShiftyLook (ns: 0)
*: Sorcery 101 (ns: 0)
*: The Soxaholix 

**: Category:Finnish webcomics (ns: 14)
*: Category:Lists of webcomics (ns: 14)
**: Lists of webcomics (ns: 0)
**: List of webcomics with LGBT characters (ns: 0)
**: List of webcomics in print (ns: 0)
**: List of video game webcomics (ns: 0)
*: Category:Webcomic media (ns: 14)
**: Category:Webcomic logos (ns: 14)
**: Category:Webcomic panels (ns: 14)
**: File:Inverloch volume 01.jpeg (ns: 6)
**: File:Megatokyo vol1 1st edition.jpg (ns: 6)
**: File:NarbonicVolume1Cover.jpg (ns: 6)
**: File:Orderofthestick Newer.gif (ns: 6)
**: File:PAA game teaser.jpg (ns: 6)
**: File:Samandfuzzy2.png (ns: 6)
**: File:Schlock Mercenary book 1 - Under New Management.jpeg (ns: 6)
**: File:Zap! in Space volume 1.jpeg (ns: 6)
*: Category:Webcomics in print (ns: 14)
**: List of webcomics in print (ns: 0)
**: A Man and His Cat (ns: 0)
**: Accomplishments of the Duke's Daughter (ns: 0)
**: Achewood (ns: 0)
**: Adachi to Shimamura (ns: 0)
**: The Adventures of Dr. McNinja (ns: 0)
**: Aesthetica of a Rogue Hero 

**: Act-i-vate (ns: 0)
**: Cool Beans World (ns: 0)
**: Dark Horse Comics (ns: 0)
**: Inkblazers (ns: 0)
**: Keenspot (ns: 0)
**: Komikwerks (ns: 0)
**: Line Webtoon (ns: 0)
**: Piccoma (ns: 0)
**: Plan Nine Publishing (ns: 0)
**: Seven Seas Entertainment (ns: 0)
**: ShiftyLook (ns: 0)
**: Shueisha (ns: 0)
**: Smith Magazine (ns: 0)
**: Tapas (syndicate) (ns: 0)
**: TopatoCo (ns: 0)
**: Category:CMX (comics) (ns: 14)
**: Category:Comic Genesis (ns: 14)
**: Category:Free webcomic hosting services (ns: 14)
**: Category:Keenspot (ns: 14)
**: Category:Webtoon publishing companies (ns: 14)
*: Category:Webcomic soundtracks (ns: 14)
**: Music of Homestuck (ns: 0)
*: Category:Webcomic syndicates (ns: 14)
**: ComicFesta (ns: 0)
**: ComicSpace (ns: 0)
**: Delitoon (ns: 0)
**: Double Fine (ns: 0)
**: Double Fine Comics (ns: 0)
**: Girlamatic (ns: 0)
**: Modern Tales (ns: 0)
**: Serializer.net (ns: 0)
**: Tapas (syndicate) (ns: 0)
**: Webcomics Nation (ns: 0)
**: Category:Blank Label Comics (ns: 1

In [31]:
class Document:
    
    def __init__(self, title, text, source, category=category):

        self.category = category
        self.title = title
        self.text = text
        self.source = source
        
        self.body = {"title": self.title,
                     "text": self.text,
                     "source":self.source}

    def insert(self):
        
        slug = slugify(self.category)

        try:
            res = client.index(index=slug, body=self.body)

        except Exception as error:
            print(f"Could not create a JSON entry for an article {self.source}")

In [35]:
test_data = {}

for key in cat.categorymembers.keys():
    page = wiki_wiki.page(key)
    
    if not "Category:" in page.title:
        doc = Document(page.title, page.text, page.fullurl, category)
        print(doc.title)
#         doc.insert()

History of webcomics
Webcomic
Angry Little Girls
Boxcar Comics
Bruno (webcomic)
Business of webcomics
Chopping Block
Katie Cook (writer)
Demon-cratic Singapore
Diesel Sweeties
Dork Tower
Dumbrella
The Erotic Adventures of Buffy and Evil Vampire Willow
Everyday Blues
Explodingdog
Game Critter Super-Squad!
Gender and webcomics
Goats (webcomic)
Guest comic
HJ-Story
Inkblazers
Keenspot
Marvel CyberComics
Modern Polaxis
New England Webcomics Weekend
Once Upon a Saturday
Platinum Grit
The Really Heavy Greatcoat
Roger Cosmonkey
Rogues of Clwyd-Rhan
Saturnalia (webcomic)
Savage Chickens
ShiftyLook
Sorcery 101
The Soxaholix
Sunstone (comics)
Surviving the World
Terminal Lance
Thinking Ape Blues
Time (xkcd)
Toonlet
Triangle and Robert
Two Lumps
Wumo
You Damn Kid!
Zuda Comics


In [33]:
# Example document search:

def search(text, index=''):
    return client.search(index=index, 
                         body={"query": 
                                 {"match": 
                                  {"text": text}
                                 }
                                }
    )

In [34]:
def test_search():
    res = search(text='xkcd')

    print('Relevant articles:')
    print('===================')
    for doc in res['hits']['hits']:
        print(doc['_source']['title'])

test_search()

Relevant articles:
Time (xkcd)
